In [2]:
using CUDA
using BenchmarkTools

## Atomic
useful when multiple threads try to write to the same global memory. prevent race condition

In [13]:
function increment_counter(counter)
    idx = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    if idx <= 100
        counter[1] += 1
    end
    return
end

counter = CuArray([0])

threads = 100
blocks = 1

@cuda threads=threads blocks=blocks increment_counter(counter)
result = Array(counter)
println("accumulated result: ", result[1])


accumulated result: 1


## parallel block reduction

In [11]:
function reduce_atomic(op,a,b)
    i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    CUDA.@atomic b[] = op(b[], a[i])
    return
end

a = CuArray(1:16)
b = CuArray([0])
@cuda threads=length(a) reduce_atomic(+, a, b)
CUDA.@allowscalar b[]

136

In [20]:
function reduce_block(op, a, b)
    #each thread takes care of 2 elements
    elements = blockDim().x * 2 
    # @cuprintf("Elements: %d\n", elements)
    thread = threadIdx().x 

    d = 1 #distance between elements
    while d < elements
        sync_threads() 
        index = 2 * d * (thread-1) + 1
        if index <= elements && index+d <= length(a)
            @inbounds a[index] = op(a[index], a[index+d])
        end
        d *= 2
    end

    if thread == 1
        b[1] = a[1]
    end
    return
end

a = CuArray(1:16)  
b = CuArray([0])   

threads = cld(length(a),2)  
@cuda threads=threads reduce_block(+, a, b) 

result = Array(b)
println("reduced sum result: ", result[1])

reduced sum result: 136


it's fine to have loops in gpu code but make sure every iteration the kernel function behaves identically   
gpu performs the best when threads are executing the same operation.  
bad case --> thread divergence  

### shared memory
access to shared memory between threads is faster than access to global memory

In [24]:
function reduce_grid_atomic_shmem(op, a, b)
    elements = blockDim().x * 2
    thread = threadIdx().x
    block = blockIdx().x
    offset = (block-1) * elements


    shared = @cuStaticSharedMem(Int32, (2048,))
    @inbounds shared[thread+blockDim().x] = a[offset+thread+blockDim().x]
    @inbounds shared[thread] = a[offset+thread]

    d = 1
    while d < elements
        sync_threads()
        index = 2 * d * (thread-1) + 1
        @inbounds if index < elements && offset+index+d <= length(a)
            shared[index] = op(shared[index], shared[index+d])
        end
        d *= 2
    end

    if thread == 1
        CUDA.@atomic b[] = op(b[], shared[1])
    end

    return
end

a = CuArray(1:16) 
b = CuArray([0]) 

@cuda threads=4 blocks=2 reduce_grid_atomic_shmem(+, a, b)
CUDA.@allowscalar b[]

136